In [1]:
from sympy import Derivative, Function, Matrix, Symbol, symbols

In [2]:
def replace_multiple(text, old_list, new_list):
    for old, new in zip(old_list, new_list):
        text = text.replace(old, new)
    return text

In [30]:
x, dt, y, t = symbols('x dt y t')

In [31]:
h = Function('h')(x, y, t)
u0 = Function('u0')(x)
u1 = Function('u1')(x)
w0 = Function('w0')(x)
b = Function('b')(x)
oldu0 = Function('oldu0')(x)
oldu1 = Function('oldu1')(x)
oldw0 = Function('oldw0')(x)
oldw1 = Function('oldw1')(x)
p0 = Function('p0')(x)
p1 = Function('p1')(x)


In [5]:
tau = Matrix([[0, 
               (h * p0).diff(x) + 2*p1*b.diff(x),
               (h * p1).diff(x) -(3*p0 - p1)*h.diff(x) - 6*(p0-p1) * b.diff(x),
                - 2*p1,
               6 * (p0-p1)
              ]]).T

In [6]:
Uk = Matrix([[ h, h * oldu0, h*oldu1, h*oldw0, h*oldw1]]).T

In [7]:
U = Uk - dt * tau

In [8]:
h = U[0]
u0 = U[1]/h
u1 = U[2]/h
w0 = U[3]/h
w1 = U[4]/h

In [37]:
I1 = h*u0.diff(x) + 1/3 * (h*u1).diff(x) + 1/3 * u1 * h.diff(x) + 2*(w0 - u0 * b.diff(x) + h.diff(t).diff(x).diff(y))

In [38]:
I1

h(x, y, t)*Derivative(u0(x), x) + 0.333333333333333*h(x, y, t)*Derivative(u1(x), x) - 2*u0(x)*Derivative(b(x), x) + 0.666666666666667*u1(x)*Derivative(h(x, y, t), x) + 2*w0(x) + 2*Derivative(h(x, y, t), t, x, y)

In [39]:
def make_derivative_symbols(expr, variable_order=None):
    """
    Builds a mapping from Derivative(...) to Symbol(...) using standardized naming,
    handling multiple variables and arbitrary derivative orders.

    Parameters:
        expr: sympy expression
        variable_order: Optional list of sympy Symbols defining variable order, e.g., [t, x, y, z]

    Returns:
        subs_dict: dict mapping Derivative(...) → Symbol('dFUNCdxdydz...')
    """
    subs_dict = {}

    # Detect all variables in derivatives if not provided
    if variable_order is None:
        variables = sorted({v for d in expr.atoms(Derivative) for v in d.variables}, key=str)
    else:
        variables = variable_order

    for d in sorted(expr.atoms(Derivative), key=str):
        f = d.expr.func.__name__  # function name like 'h' or 'p0'
        counts = {v: 0 for v in variables}

        for v in d.variables:
            counts[v] += 1

        # Build the derivative name like dFUNCddxdydz
        name = 'd' + f
        for v in variables:
            if counts[v] > 0:
                name += 'd' * counts[v] + v.name  # e.g., ddx, dy

        subs_dict[d] = Symbol(name)

    return subs_dict

In [40]:
derivs = sorted(I1.atoms(Derivative), key=str)
derivs

[Derivative(b(x), x),
 Derivative(h(x, y, t), t, x, y),
 Derivative(h(x, y, t), x),
 Derivative(u0(x), x),
 Derivative(u1(x), x)]

In [41]:
make_derivative_symbols(I1)

{Derivative(b(x), x): dbdx,
 Derivative(h(x, y, t), t, x, y): dhdtdxdy,
 Derivative(h(x, y, t), x): dhdx,
 Derivative(u0(x), x): du0dx,
 Derivative(u1(x), x): du1dx}

In [28]:
replace_list = [str(d) for d in derivs] 
subs_list = ['ddbdxx', 'dbdx', 'ddhdxx', 'dhdx', 'doldu0dx', 'doldu1dx', 'ddp0dxx', 'dp0dx', 'ddp1dxx', 'dp1dx']

In [29]:
out = replace_multiple(str(I1), replace_list, subs_list)
replace_multiple(out, ['(x)'], [''])

'-0.333333333333333*dt*(-(3*p0 - p1)*ddhdxx - (6*p0 - 6*p1)*ddbdxx - (3*dp0dx - dp1dx)*dhdx - (6*dp0dx - 6*dp1dx)*dbdx + h*ddp1dxx + p1*ddhdxx + 2*dhdx*dp1dx) - 2*(-dt*(h*dp0dx + p0*dhdx + 2*p1*dbdx) + h*oldu0)*dbdx/h + 0.333333333333333*(-dt*(-(3*p0 - p1)*dhdx - (6*p0 - 6*p1)*dbdx + h*dp1dx + p1*dhdx) + h*oldu1)*dhdx/h + 2*(2*dt*p1 + h*oldw0)/h + (-(-dt*(h*dp0dx + p0*dhdx + 2*p1*dbdx) + h*oldu0)*dhdx/h**2 + (-dt*(h*ddp0dxx + p0*ddhdxx + 2*p1*ddbdxx + 2*dbdx*dp1dx + 2*dhdx*dp0dx) + h*doldu0dx + oldu0*dhdx)/h)*h + 0.333333333333333*h*doldu1dx + 0.333333333333333*oldu1*dhdx'

In [30]:
I2 = h * u0.diff(x) + u1*h.diff(x) + 2*(u1*b.diff(x) - w1)

In [31]:
derivs = sorted(I2.atoms(Derivative), key=str)
derivs

[Derivative(b(x), (x, 2)),
 Derivative(b(x), x),
 Derivative(h(x), (x, 2)),
 Derivative(h(x), x),
 Derivative(oldu0(x), x),
 Derivative(p0(x), (x, 2)),
 Derivative(p0(x), x),
 Derivative(p1(x), x)]

In [32]:
replace_list = [str(d) for d in derivs]
subs_list = ['ddbdxx', 'dbdx', 'ddhdxx', 'dhdx', 'doldu0dx', 'ddp0dxx', 'dp0dx', 'dp1dx']

In [33]:
out = replace_multiple(str(I2), replace_list, subs_list)
replace_multiple(out, ['(x)'], [''])

'-2*(-dt*(6*p0 - 6*p1) + h*oldw1)/h + 2*(-dt*(-(3*p0 - p1)*dhdx - (6*p0 - 6*p1)*dbdx + h*dp1dx + p1*dhdx) + h*oldu1)*dbdx/h + (-dt*(-(3*p0 - p1)*dhdx - (6*p0 - 6*p1)*dbdx + h*dp1dx + p1*dhdx) + h*oldu1)*dhdx/h + (-(-dt*(h*dp0dx + p0*dhdx + 2*p1*dbdx) + h*oldu0)*dhdx/h**2 + (-dt*(h*ddp0dxx + p0*ddhdxx + 2*p1*ddbdxx + 2*dbdx*dp1dx + 2*dhdx*dp0dx) + h*doldu0dx + oldu0*dhdx)/h)*h'